# Applied Data Science Capstone Week 4&5

Capstone Project - The Battle of Neighborhoods

Discovering Top Venues in Scarborough, Toronto, Ontario, Canada

In [1]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

from bs4 import BeautifulSoup

import xml

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

In [3]:
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighborhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighborhood.append(fields[i+2].text.strip())
        
df_pc = pd.DataFrame(data=[postcode, borough, neighborhood]).transpose()
df_pc.columns = ['Postcode', 'Borough', 'Neighborhood']

not_assigned_boroughs = df_pc.index[df_pc['Borough'] == 'Not assigned']
not_assigned_neighborhoods = df_pc.index[df_pc['Neighborhood'] == 'Not assigned']
for idx in not_assigned_neighborhoods:
    df_pc['Neighborhood'][idx] = df_pc['Borough'][idx]
    
df_pc['Borough'].replace('Not assigned', np.nan, inplace=True)
df_pc.dropna(subset=['Borough'], inplace=True)

df_pcn = df_pc.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_pcn.columns = ['Postcode', 'Borough', 'Neighborhood']

df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns = ['Postcode', 'Latitude', 'Longitude']

df_pos = pd.merge(df_pcn, df_geo, on=['Postcode'], how='inner')

df_tor = df_pos[['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']].copy()

df_sc = df_tor[df_tor['Borough'].str.contains('Scarborough')]

In [4]:
df_sc

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [6]:
address = 'Scarborough, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_neighbor = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(df_sc['Latitude'], df_sc['Longitude'], df_sc['Borough'], df_sc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_neighbor)  
    
map_neighbor

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


In [7]:
CLIENT_ID = 'HIV1Q1KKDIYXMR4E1NIYJZS34RCQOPLNZKJP5L2CBSLJ1ZXW'
CLIENT_SECRET = 'CT4UAW5Y5MHKOHUYPY0SGA0LXDV1ZYKLR5NON5CVWLSG4LA2'
VERSION = '20190811'

In [8]:
def foursquare_crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 200, radius = 500):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result_ds;

In [9]:
print('Crawling different neighborhoods inside "Scarborough"')
SC_foursquare_dataset = foursquare_crawler(list(df_sc['Postcode']),
                                                   list(df_sc['Neighborhood']),
                                                   list(df_sc['Latitude']),
                                                   list(df_sc['Longitude']),)

Crawling different neighborhoods inside "Scarborough"
1.
Data is Obtained, for the Postal Code M1B (and Neighborhoods Rouge, Malvern) SUCCESSFULLY.
2.
Data is Obtained, for the Postal Code M1C (and Neighborhoods Highland Creek, Rouge Hill, Port Union) SUCCESSFULLY.
3.
Data is Obtained, for the Postal Code M1E (and Neighborhoods Guildwood, Morningside, West Hill) SUCCESSFULLY.
4.
Data is Obtained, for the Postal Code M1G (and Neighborhoods Woburn) SUCCESSFULLY.
5.
Data is Obtained, for the Postal Code M1H (and Neighborhoods Cedarbrae) SUCCESSFULLY.
6.
Data is Obtained, for the Postal Code M1J (and Neighborhoods Scarborough Village) SUCCESSFULLY.
7.
Data is Obtained, for the Postal Code M1K (and Neighborhoods East Birchmount Park, Ionview, Kennedy Park) SUCCESSFULLY.
8.
Data is Obtained, for the Postal Code M1L (and Neighborhoods Clairlea, Golden Mile, Oakridge) SUCCESSFULLY.
9.
Data is Obtained, for the Postal Code M1M (and Neighborhoods Cliffcrest, Cliffside, Scarborough Village West) 

In [10]:
def get_venue_dataset(foursquare_dataset):
    result_df = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])
    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Postal Code']; neigh = neigh_dict['Neighborhood(s)']
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        print('Number of Venuse in Coordination "{}" Posal Code and "{}" Negihborhood(s) is: {}.'.format(postal_code, neigh, len(neigh_dict['Crawling_result'])))
        
        for venue_dict in neigh_dict['Crawling_result']:
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            dist = venue_dict['venue']['location']['distance']
            cat =  venue_dict['venue']['categories'][0]['name']
            
            
            result_df = result_df.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)
    
    return(result_df)

In [11]:
sc_venues = get_venue_dataset(SC_foursquare_dataset)

Number of Venuse in Coordination "M1B" Posal Code and "Rouge, Malvern" Negihborhood(s) is: 1.
Number of Venuse in Coordination "M1C" Posal Code and "Highland Creek, Rouge Hill, Port Union" Negihborhood(s) is: 2.
Number of Venuse in Coordination "M1E" Posal Code and "Guildwood, Morningside, West Hill" Negihborhood(s) is: 7.
Number of Venuse in Coordination "M1G" Posal Code and "Woburn" Negihborhood(s) is: 4.
Number of Venuse in Coordination "M1H" Posal Code and "Cedarbrae" Negihborhood(s) is: 8.
Number of Venuse in Coordination "M1J" Posal Code and "Scarborough Village" Negihborhood(s) is: 2.
Number of Venuse in Coordination "M1K" Posal Code and "East Birchmount Park, Ionview, Kennedy Park" Negihborhood(s) is: 4.
Number of Venuse in Coordination "M1L" Posal Code and "Clairlea, Golden Mile, Oakridge" Negihborhood(s) is: 9.
Number of Venuse in Coordination "M1M" Posal Code and "Cliffcrest, Cliffside, Scarborough Village West" Negihborhood(s) is: 2.
Number of Venuse in Coordination "M1N" P

In [12]:
sc_venues['Venue Category'].unique()

array(['Fast Food Restaurant', 'Bar', 'Moving Target', 'Pizza Place',
       'Electronics Store', 'Mexican Restaurant', 'Rental Car Location',
       'Medical Center', 'Intersection', 'Breakfast Spot', 'Coffee Shop',
       'Korean Restaurant', 'Pharmacy', 'Hakka Restaurant',
       'Caribbean Restaurant', 'Thai Restaurant', 'Athletics & Sports',
       'Bank', 'Bakery', 'Fried Chicken Joint', 'Lounge', 'Playground',
       'Convenience Store', 'Department Store', 'Chinese Restaurant',
       'Hobby Shop', 'Bus Line', 'Metro Station', 'Bus Station', 'Park',
       'Soccer Field', 'Motel', 'American Restaurant', 'Café',
       'General Entertainment', 'Skating Rink', 'College Stadium',
       'Indian Restaurant', 'Vietnamese Restaurant', 'Pet Store',
       'Latin American Restaurant', 'Smoke Shop',
       'Middle Eastern Restaurant', 'Clothing Store',
       'Italian Restaurant', 'Noodle House', 'Sandwich Place',
       'Grocery Store', 'Cosmetics Shop', 'Bubble Tea Shop', 'Nail Salon'

In [13]:
sc_re = sc_venues.loc[sc_venues['Venue Category'].isin(sc_venues['Venue Category'].unique())]

In [14]:
sc_re

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M1B,"Rouge, Malvern",43.806686,-79.194353,Wendy's,This spot is popular,Fast Food Restaurant,387
1,M1C,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,This spot is popular,Bar,304
2,M1C,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,This spot is popular,Moving Target,426
3,M1E,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,This spot is popular,Pizza Place,469
4,M1E,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,This spot is popular,Electronics Store,298
5,M1E,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,This spot is popular,Mexican Restaurant,343
6,M1E,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,This spot is popular,Rental Car Location,381
7,M1E,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,This spot is popular,Medical Center,445
8,M1E,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,This spot is popular,Intersection,464
9,M1E,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Eggsmart,This spot is popular,Breakfast Spot,491


Now let us figure out how popular/common these venues are.

In [15]:
sc_onehot = pd.get_dummies(sc_re[['Venue Category']], prefix="", prefix_sep="")

sc_onehot['Neighborhood'] = sc_re['Neighborhood'] 

fixed_columns = [sc_onehot.columns[-1]] + list(sc_onehot.columns[:-1])
sc_onehot = sc_onehot[fixed_columns]

sc_grouped = sc_onehot.groupby('Neighborhood').mean().reset_index()

sc_onehot = pd.get_dummies(data = sc_re, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])

sc_onehot = sc_onehot.groupby('Neighborhood').sum()

kmeans = KMeans(n_clusters = 5, random_state = 0).fit(sc_onehot)

means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = sc_onehot.columns
means_df.index = ['G1','G2','G3','G4','G5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

neigh_summary = pd.DataFrame([sc_onehot.index, 1 + kmeans.labels_]).T
neigh_summary.columns = ['Neighborhood', 'Group']
neigh_summary

,Neighborhood,Group
0,Agincourt,4
1,"Agincourt North, L'Amoreaux East, Milliken, St...",1
2,"Birch Cliff, Cliffside West",4
3,Cedarbrae,2
4,"Clairlea, Golden Mile, Oakridge",5
5,"Clarks Corners, Sullivan, Tam O'Shanter",5
6,"Cliffcrest, Cliffside, Scarborough Village West",1
7,"Dorset Park, Scarborough Town Centre, Wexford ...",2
8,"East Birchmount Park, Ionview, Kennedy Park",4
9,"Guildwood, Morningside, West Hill",2


In [16]:
num_top_venues = 10

for hood in sc_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = sc_grouped[sc_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1             Breakfast Spot  0.25
2               Skating Rink  0.25
3             Clothing Store  0.25
4        American Restaurant  0.00
5               Noodle House  0.00
6  Latin American Restaurant  0.00
7             Medical Center  0.00
8              Metro Station  0.00
9         Mexican Restaurant  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                       venue  freq
0                       Park   0.5
1                 Playground   0.5
2               Noodle House   0.0
3          Korean Restaurant   0.0
4  Latin American Restaurant   0.0
5                     Lounge   0.0
6             Medical Center   0.0
7              Metro Station   0.0
8         Mexican Restaurant   0.0
9  Middle Eastern Restaurant   0.0


----Birch Cliff, Cliffside West----
                       venue  freq
0               Skating Rink  0.25
1      General Entertainment  0.25
2 

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sc_grouped['Neighborhood']

for ind in np.arange(sc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sc_grouped.iloc[ind, :], num_top_venues)

In [19]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Cosmetics Shop
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Lounge,Caribbean Restaurant,Hakka Restaurant,Convenience Store,Grocery Store
4,"Clairlea, Golden Mile, Oakridge",Bus Line,Soccer Field,Bakery,Intersection,Fast Food Restaurant,Bus Station,Park,Metro Station,Vietnamese Restaurant,Coffee Shop
5,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Fried Chicken Joint,Thai Restaurant,Bank,Fast Food Restaurant,Italian Restaurant,Pharmacy,Noodle House,Chinese Restaurant,Vietnamese Restaurant
6,"Cliffcrest, Cliffside, Scarborough Village West",American Restaurant,Motel,Hobby Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Cosmetics Shop
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Pet Store,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Sandwich Place,Soccer Field,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
8,"East Birchmount Park, Ionview, Kennedy Park",Chinese Restaurant,Department Store,Coffee Shop,Hobby Shop,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
9,"Guildwood, Morningside, West Hill",Intersection,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Fried Chicken Joint


In [20]:
df_scre = df_sc.loc[df_sc['Neighborhood'].isin(df_sc['Neighborhood'].unique())]

map_neighbor1 = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(df_scre['Latitude'], df_scre['Longitude'], 
                                            df_scre['Borough'], df_scre['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_neighbor1)  
    
map_neighbor1